## Custom Chatbot Project

# Table of Contents
- [Dataset](#Dataset:-2023_fashion_trends.csv)
- [Data Wrangling](#Data-Wrangling)
- [Custom Query Completion](#Custom-Query-Completion)
- [Custom Prompt Demonstration](#Custom-Prompt-Demonstration)
- [Basic Prompt Demonstration](#Basic-Prompt-Demonstration)
- [Chatbot](#chatbot)

### Dataset: 2023_fashion_trends.csv

This dataset provides insights into the fashion trends for the year 2023. This dataset contains 94 rows. Each row contains the URL, Trends,and Source. 

I will tranform the dataset to include two columns to create a custom text column in a textual narrative form.

For example:

text = Source of the Trend. Trend Description.

The text column from the transformed dataset will be used to build custom chatbot.

Use case scenario:
The primary goal of this chatbot is to provide users with information and insights about the latest fashion trends for 2023. It aims to assist fashion enthusiasts, shoppers, and industry professionals in staying updated with the current trends and making informed fashion choices.

## Data Wrangling

In [17]:
import pandas as pd

In [18]:
df = pd.read_csv('./2023_fashion_trends.csv')
df.head()

,URL,Trends,Source
0,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Red. Glossy red hues took ...,7 Fashion Trends That Will Take Over 2023 — Sh...
1,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Cargo Pants. Utilitarian w...,7 Fashion Trends That Will Take Over 2023 — Sh...
2,https://www.refinery29.com/en-us/fashion-trend...,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...",7 Fashion Trends That Will Take Over 2023 — Sh...
3,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Denim Reimagined. From dou...,7 Fashion Trends That Will Take Over 2023 — Sh...
4,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Shine For The Daytime. The...,7 Fashion Trends That Will Take Over 2023 — Sh...


In [19]:
df['text'] = df['Source'] + ": " + df['Trends'] + "."
df['text'][0] 

'7 Fashion Trends That Will Take Over 2023 — Shop Them Now: 2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that would look just as good in head-to-toe looks (see: a pantsuit) as accent accessory pieces (shoes, handbags, jewelry)..'

In [29]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI()

EMBEDDINGS_MODEL_NAMES = "text-embedding-3-small"

embeddings = []
for text in df['text']:
    response = client.embeddings.create(
        input=text,
        model=EMBEDDINGS_MODEL_NAMES
    )
    embeddings.append(response.data[0].embedding)

df['embeddings'] = embeddings
df.head()

,text,embeddings
0,7 Fashion Trends That Will Take Over 2023 — Sh...,"[-0.007465763948857784, 0.016444619745016098, ..."
1,7 Fashion Trends That Will Take Over 2023 — Sh...,"[0.004193933680653572, 0.04125695303082466, 0...."
2,7 Fashion Trends That Will Take Over 2023 — Sh...,"[0.061038773506879807, 0.016439247876405716, 0..."
3,7 Fashion Trends That Will Take Over 2023 — Sh...,"[0.025815743952989578, 0.032582979649305344, 0..."
4,7 Fashion Trends That Will Take Over 2023 — Sh...,"[-0.0011280509643256664, -0.009036949835717678..."


## Custom Query Completion

In [ ]:
def question_embeddings(question):
  response = client.embeddings.create(
      input=question,
      model=EMBEDDINGS_MODEL_NAMES
  )
  return response.data[0].embedding

In [34]:
from typing import List
from scipy import spatial


def distances_from_embeddings(query_embedding, embeddings):
  return [spatial.distance.cosine(query_embedding, emb) for emb in embeddings]


# generate prompts
def get_prompt(question, df):
  query_embedding = question_embeddings(question)
  df['distances'] = distances_from_embeddings(query_embedding, df['embeddings'])
  df_sorted = df.sort_values(by='distances')
  context = "\n\n###\n\n".join(df_sorted['text'].head(5))
  prompt = f"""
Answer the question based on the context below, and if the
question can't be answered based on the context, say
"I don't know"

Context:

{context}

---

Question: {question}
Answer:"""
  return prompt


In [35]:
# Test questions
q1 = "What are the key fashion trends for 2023?"
q2 = "How can I incorporate cobalt blue into my wardrobe?"

In [36]:
q1_prompt=get_prompt(q1, df)
q2_prompt=get_prompt(q2, df)
print(q1, q2)
print('-------------------------')
print(q1_prompt, q2_prompt)

What are the key fashion trends for 2023? How can I incorporate cobalt blue into my wardrobe?
-------------------------

Answer the question based on the context below, and if the
question can't be answered based on the context, say
"I don't know"

Context:

Spring/Summer 2023 Fashion Trends: 21 Expert-Approved Looks You Need to See: In a diplomatic turn, designers have extended the lifespan of some noteworthy A/W 22 trends, continuing their exploration of them in 2023, ensuring we get adequate wear out of the pieces in which we've already invested. Elevated basics is one such trend. Last season, there was a clear '90s directive when it came to minimalism, with logo vests, low-slung trousers and tube dresses being hero items. Now, things have been pared back even further; sure, Y2K maxi skirts and oversized suiting are very much a thing, as witnessed on the runways of Peter Do and Roksanda respectively, but the silhouettes are cleaner and the colour palette looks more expensive; think 

## Custom Prompt Demonstration

In [37]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo"
response = client.chat.completions.create(
    model=COMPLETION_MODEL_NAME,
    messages = [
        {
            "role": "user",
            "content": q1_prompt
        }
    ],
    max_tokens=150
)
print(response.choices[0].message.content)

Some key fashion trends for 2023 include elevated basics, Y2K maxi skirts, oversized suiting, slouchy-fit trousers, wide-leg and puddle hemlines, looser-fit denim, sheer clothing, and 3D designs with floral motifs.


In [38]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo"
response = client.chat.completions.create(
    model=COMPLETION_MODEL_NAME,
    messages = [
        {
            "role": "user",
            "content": q2_prompt
        }
    ],
    max_tokens=150
)
print(response.choices[0].message.content)

You can incorporate cobalt blue into your wardrobe by adding pieces like clothing, shoes, or accessories in that color. It pairs well with other colors like Pantone's Viva Magenta.


## Basic Prompt Demonstration

In [39]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo"
response = client.chat.completions.create(
    model=COMPLETION_MODEL_NAME,
    messages = [
        {
            "role": "user",
            "content": q1
        }
    ],
    max_tokens=150
)
print(response.choices[0].message.content)

1. Sustainability: The focus on eco-friendly and sustainable fashion will continue to be a key trend in 2023, with more brands and designers incorporating sustainable practices into their collections.

2. Gender neutral fashion: Gender fluidity and inclusivity will be emphasized in fashion in 2023, with more brands offering gender-neutral clothing options and blurring traditional gender lines.

3. Bold and vibrant colors: Bright and bold colors will be a major trend in 2023, with neon shades, vibrant pinks, blues, and greens making a statement in fashion collections.

4. Maximalism: Over-the-top, extravagant designs and embellishments will be popular in 2023, as fashion embraces bold prints, textures, and patterns.

5. Vintage


In [40]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo"
response = client.chat.completions.create(
    model=COMPLETION_MODEL_NAME,
    messages = [
        {
            "role": "user",
            "content": q2
        }
    ],
    max_tokens=150
)
print(response.choices[0].message.content)

1. Start with accessories: Add a pop of cobalt blue to your outfits with accessories such as shoes, handbags, scarves, or statement jewelry.

2. Mix and match: Pair cobalt blue pieces with neutral colors like black, white, or grey for a sophisticated look. Alternatively, you can create a bold and colorful outfit by combining cobalt blue with other bright hues like yellow or pink.

3. Go for a cobalt blue statement piece: Invest in a cobalt blue blazer, dress, or coat that can be the focal point of your outfit. This will instantly elevate your look and add a touch of vibrancy to your wardrobe.

4. Experiment with different textures: Try incorporating cobalt blue in different fabrics like silk


### Chatbot

In [43]:
def get_chatbot_response(prompt):
    response = client.chat.completions.create(
        model=COMPLETION_MODEL_NAME,
        messages = [
            {
                "role": "user",
                "content": prompt
            }
        ],
        max_tokens=150
    )
    return response.choices[0].message.content

while True:
    user_question = input("Ask a question about fashion trends (or type 'exit' to quit)")
    if user_question.lower() == 'exit':
        print('Goodbye!')
        break

    custom_prompt = get_prompt(user_question, df)
    response = get_chatbot_response(custom_prompt)
    print("Chatbot:", response)

Chatbot: The key fashion trends for 2023 include elevated basics, looser-fit denim, sheer clothing, and surrealistic designs with 3D elements and floral motifs.
Chatbot: Based on the context provided, the denim trends for 2023 include denim reimagined with double-waisted jeans and carpenter jeans, baggy denim, wide-legs and puddle hemlines, slouchy-fit trousers, denim-on-denim looks, and a return to more relaxed silhouettes and looser-fit denim styles.
Chatbot: Cobalt blue, red, lime green, pastel shades like lilac, pale yellow, and baby blue.
Chatbot: You can incorporate cobalt blue into your wardrobe by choosing clothing items in that color such as tops, bottoms, dresses, or accessories like bags or shoes. You can also pair cobalt blue pieces with complementary colors like white, black, or magenta for a stylish look.
Chatbot: Some ways to style cargo pants this year include pairing them with tailored silhouettes, interesting pocket placements, elevated fabrics like silk and organza, 